## CESM2 - LARGE ENSEMBLE (LENS2)

- The goal of this notebook is to calculate the MOC in density coordinates. We used https://github.com/sgyeager/POP_MOC as reference; thus, We thank Dr. Stephen Yeager. We also thank Michael Levy for the technical support. 
- P.S.: The Notebook is incomplete as we are adapting it to compute the MOC from all its components. 

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2
import xarray as xr 
import numpy as np  
import cftime
import copy
import scipy.stats
from scipy import signal
from functools import partial
import glob
import dask
import cf_xarray
import intake
import pprint
import intake_esm
import matplotlib.pyplot as plt
from xhistogram.xarray import histogram
import pop_tools
%matplotlib inline
from MOCutils import popmoc
from dask.distributed import Client
from ncar_jobqueue import NCARCluster#,PBSCluster

### Improve the workflow using clusters 

In [ ]:
mem_per_worker = 100 # in GB more memory here maybe 100 GB
num_workers = 45 # more workers maybe 45
cluster = NCARCluster(cores=1, processes=1, memory=f'{mem_per_worker} GB',resource_spec=f'select=1:ncpus=1:mem={mem_per_worker}GB')
cluster.scale(num_workers)
client = Client(cluster)
print(client)
client

### Read in OGCM history file & MOC template file

In [ ]:
fmoc = '/glade/u/home/yeager/analysis/python/POP_MOC/moc_template.nc'
ds_moctemp = xr.open_dataset(fmoc) # MOC template

In [ ]:
# Open original collection description file
cat_url='/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
col = intake.open_esm_datastore(cat_url)

In [ ]:
# Catolog
print('Catalog file:', col.esmcol_data['catalog_file'])
col.df.head(10)

In [ ]:
uniques_orig = col.unique(columns=['component', 'frequency', 'experiment', 'variable'])
pprint.pprint(uniques_orig, compact=True, indent=1, width=80)

In [ ]:
# Some variables like temperature, salinity, are not available for annual frequency, so we chose the monthly frequency.
col.search(component="ocn", variable=["TEMP","SALT","UVEL","VVEL"], frequency="month_1", experiment="historical").df

In [ ]:
%%time
cat_subset = col.search(component='ocn', # Ocean component
                            variable=['TEMP','SALT','UVEL','VVEL','UISOP','USUBM','VISOP','VSUBM'], # Temperature, Salinity, Zonal Velocity, Meridional Velocity
                            frequency='month_1', # Monthly
                            experiment='historical') # 1850-2014
#                            forcing_variant='smbb', # You can use smbb or cmip6
#                       )
dset_dict_raw = cat_subset.to_dataset_dict(zarr_kwargs={"consolidated": True}, storage_options={"anon": True})
print(f"\nDataset dictionary keys:\n {dset_dict_raw.keys()}")

### Compute sigma-2 field from LENS2 dataset

In [ ]:
tslice = slice("1960-01-01", "2014-12-31") # select the period you wish

### Salinity

In [ ]:
%%time
ds_smbb_salt = dset_dict_raw['ocn.historical.pop.h.smbb.SALT'] # Salinity
#ds_cmip6_salt = dset_dict_raw['ocn.historical.pop.h.cmip6.SALT'] # Salinity
#ds_salt = xr.concat([ds_cmip6_salt,ds_smbb_salt], dim='member_id',data_vars='minimal',coords='minimal',compat='override')
print(f'Salinty before: {dask.utils.format_bytes(ds_smbb_salt.nbytes)}')
ds_salt = ds_smbb_salt.sel(time=tslice)
ds_salt = ds_salt.resample(time='1Y', closed='left').mean('time') # Yearly average
salt = ds_salt['SALT']
salt = salt.mean(dim = ["member_id"]) # Average of all members
print(f"Salinty after: {dask.utils.format_bytes(salt.nbytes)}")
salt = salt.load() # Necessary because pop-tools.eos() doesn't play nicely with dask
del ds_salt, ds_smbb_salt#, ds_cmip6_salt

### Temperature

In [ ]:
%%time
ds_smbb_temp = dset_dict_raw['ocn.historical.pop.h.smbb.TEMP'] # Temperature
#ds_cmip6_temp = dset_dict_raw['ocn.historical.pop.h.cmip6.TEMP'] # Temperature
#ds_temp = xr.concat([ds_cmip6_temp,ds_smbb_temp], dim='member_id',data_vars='minimal',coords='minimal',compat='override')
print(f'Temperature before: {dask.utils.format_bytes(ds_smbb_temp.nbytes)}')
ds_temp = ds_smbb_temp.sel(time=tslice)
ds_temp = ds_temp.resample(time='1Y', closed='left').mean('time') # Yearly average
temp = ds_temp['TEMP']
temp = temp.mean(dim = ['member_id']) # Average of all members
print(f'Temperature after: {dask.utils.format_bytes(temp.nbytes)}')
temp = temp.load() # Necessary because pop-tools.eos() doesn't play nicely with dask
del ds_smbb_temp#, ds_cmip6_temp

### Zonal velocity

In [ ]:
%%time
ds_smbb_uvel = dset_dict_raw['ocn.historical.pop.h.smbb.UVEL'] #  Zonal velocity
#ds_cmip6_uvel = dset_dict_raw['ocn.historical.pop.h.cmip6.UVEL'] #  Zonal velocity
#ds_uvel = xr.concat([ds_cmip6_uvel,ds_smbb_uvel], dim='member_id',data_vars='minimal',coords='minimal',compat='override')
print(f'Zonal Velocity before: {dask.utils.format_bytes(ds_smbb_uvel.nbytes)}')
ds_uvel = ds_smbb_uvel.sel(time=tslice)
ds_uvel = ds_uvel.resample(time='1Y', closed='left').mean('time') # Yearly average
uvel = ds_uvel['UVEL']
uvel = uvel.mean(dim = ['member_id']) # Average of all members
print(f'Zonal Velocity before: {dask.utils.format_bytes(uvel.nbytes)}')
uvel = uvel.load() # Necessary because pop-tools.eos() doesn't play nicely with dask
del ds_smbb_uvel#, ds_cmip6_uvel

### Meridional Velocity

In [ ]:
%%time
ds_smbb_vvel = dset_dict_raw['ocn.historical.pop.h.smbb.VVEL'] # meridional velocity
#ds_cmip6_vvel = dset_dict_raw['ocn.historical.pop.h.cmip6.VVEL'] # meridional velocity
#ds_vvel = xr.concat([ds_cmip6_vvel,ds_smbb_vvel], dim='member_id',data_vars='minimal',coords='minimal',compat='override')
print(f"Meridional Velocity before: {dask.utils.format_bytes(ds_smbb_vvel.nbytes)}")
ds_vvel = ds_smbb_vvel.sel(time=tslice)
ds_vvel = ds_vvel.resample(time='1Y', closed='left').mean('time') # Yearly average
vvel = ds_vvel['VVEL']
vvel = vvel.mean(dim = ['member_id']) # Average of all members
print(f"Zonal Velocity before: {dask.utils.format_bytes(vvel.nbytes)}")
vvel = vvel.load() # Necessary because pop-tools.eos() doesn't play nicely with dask
del ds_smbb_vvel#, ds_cmip6_vvel

### Submeso velocity in grid-x direction (diagnostic)

In [ ]:
%%time
ds_smbb_usubm = dset_dict_raw['ocn.historical.pop.h.smbb.USUBM'] # meridional velocity
#ds_cmip6_usubm = dset_dict_raw['ocn.historical.pop.h.cmip6.USUBM'] # meridional velocity
#ds_usubm = xr.concat([ds_cmip6_usubm,ds_smbb_usubm], dim='member_id',data_vars='minimal',coords='minimal',compat='override')
print(f"Zonal Submeso Velocity before: {dask.utils.format_bytes(ds_smbb_usubm.nbytes)}")
ds_usubm = ds_smbb_usubm.sel(time=tslice)
ds_usubm = ds_usubm.resample(time='1Y', closed='left').mean('time') # Yearly average
usubm = ds_usubm['USUBM']
usubm = usubm.mean(dim = ['member_id']) # Average of all members
print(f"Zonal Velocity before: {dask.utils.format_bytes(usubm.nbytes)}")
usubm = usubm.load() # Necessary because pop-tools.eos() doesn't play nicely with dask
del ds_smbb_usubm#, ds_cmip6_usubm

### Submeso velocity in grid-y direction (diagnostic)

In [ ]:
%%time 
ds_smbb_vsubm = dset_dict_raw['ocn.historical.pop.h.smbb.VSUBM'] # meridional velocity
#ds_cmip6_vsubm = dset_dict_raw['ocn.historical.pop.h.cmip6.VSUBM'] # meridional velocity
#ds_vsubm = xr.concat([ds_cmip6_vsubm,ds_smbb_vsubm], dim='member_id',data_vars='minimal',coords='minimal',compat='override')
print(f"Meridional Submeso Velocity before: {dask.utils.format_bytes(ds_smbb_vsubm.nbytes)}")
ds_vsubm = ds_smbb_vsubm.sel(time=tslice)
ds_vsubm = ds_vsubm.resample(time='1Y', closed='left').mean('time') # Yearly average
vsubm = ds_vsubm['VSUBM']
vsubm = vsubm.mean(dim = ['member_id']) # Average of all members
print(f"Zonal Velocity before: {dask.utils.format_bytes(vsubm.nbytes)}")
vsubm = vsubm.load() # Necessary because pop-tools.eos() doesn't play nicely with dask
del ds_smbb_vsubm#, ds_cmip6_vsubm

### Bolus Velocity in grid-x direction (diagnostic)

In [ ]:
%%time
ds_smbb_uisop = dset_dict_raw['ocn.historical.pop.h.smbb.UISOP'] # meridional velocity
#ds_cmip6_uisop = dset_dict_raw['ocn.historical.pop.h.cmip6.UISOP'] # meridional velocity
#ds_uisop = xr.concat([ds_cmip6_uisop,ds_smbb_uisop], dim='member_id',data_vars='minimal',coords='minimal',compat='override')
print(f"Meridional Submeso Velocity before: {dask.utils.format_bytes(ds_smbb_uisop.nbytes)}")
ds_uisop = ds_smbb_uisop.sel(time=tslice)
ds_uisop = ds_uisop.resample(time='1Y', closed='left').mean('time') # Yearly average
uisop = ds_uisop['UISOP']
uisop = uisop.mean(dim = ['member_id']) # Average of all members
print(f"Zonal Velocity before: {dask.utils.format_bytes(uisop.nbytes)}")
uisop = uisop.load() # Necessary because pop-tools.eos() doesn't play nicely with dask
del ds_smbb_uisop#, ds_cmip6_uisop

### Bolus Velocity in grid-y direction (diagnostic)

In [ ]:
%%time
ds_smbb_visop = dset_dict_raw['ocn.historical.pop.h.smbb.VISOP'] # meridional velocity
#ds_cmip6_visop = dset_dict_raw['ocn.historical.pop.h.cmip6.VISOP'] # meridional velocity
#ds_visop = xr.concat([ds_cmip6_visop,ds_smbb_visop], dim='member_id',data_vars='minimal',coords='minimal',compat='override')
print(f"Meridional Submeso Velocity before: {dask.utils.format_bytes(ds_smbb_visop.nbytes)}")
ds_visop = ds_smbb_visop.sel(time=tslice)
ds_visop = ds_visop.resample(time='1Y', closed='left').mean('time') # Yearly average
visop = ds_visop['VISOP']
visop = visop.mean(dim = ['member_id']) # Average of all members
print(f"Zonal Velocity before: {dask.utils.format_bytes(visop.nbytes)}")
visop = visop.load() # Necessary because pop-tools.eos() doesn't play nicely with dask
del ds_smbb_visop#, ds_cmip6_visop

### 2. Compute sigma-2 field from POP model output

In [ ]:
%%time
refz = 2000
refdep = xr.full_like(salt,refz).rename('REFDEP')

# Sigma2 on model TLAT, TLONG
sigma2_T = pop_tools.eos(salt=salt,temp=temp,depth=refdep) - 1000
sigma2_T = sigma2_T.assign_attrs({'long_name':'Sigma referenced to {}m'.format(refz),'units':'kg/m^3'})

## Following may be needed for some CESM2 LENS output (simulations run in Korea):
#dims = np.shape(temp)
#nt = dims[0]
#nz = dims[1]
#ny = dims[2]
#nx = dims[3]
#kji = np.indices((nz,ny,nx))
#kindices = kji[0,:,:,:] + 1
# apply T-grid mask
#mask=kindices<=ds['KMT'].values[None,:,:]
#sigma2_T = sigma2_T.where(mask)

### 3. Define target sigma-2 vertical grid
#### * Use a predefined target grid, or create your own!

In [ ]:
# Use predefined 86-layer sigma2 grid:
sigma_mid,sigma_edge = popmoc.sigma2_grid_86L()

In [ ]:
sigma_mid

In [ ]:
sigma_edge

### 4. Compute Isopycnal Layer Thickness

In [ ]:
# Here, test histogram by counting cells in each density bin. Vertical sum should be same as KMT.
iso_count = histogram(sigma2_T, bins=[sigma_edge.values],dim=['z_t'],density=False)
iso_count = iso_count.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})

kmtdiff = iso_count.sum('sigma') - ds_temp['KMT']
print("Max difference from true KMT = {}".format(abs(kmtdiff).max().values))

In [ ]:
# Use histogram to compute layer thickness. Vertical sum should be same as HT.
dzwgts = (ds_temp['dz']/100.).assign_attrs({'units':'m'})
iso_thick = histogram(sigma2_T, bins=[sigma_edge.values], weights=dzwgts,dim=['z_t'],density=False)
iso_thick = iso_thick.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
iso_thick = iso_thick.rename('iso_thick').assign_attrs({'units':'m','long_name':'Isopycnal Layer Thickness'}).rename({'sigma':'sigma_mid'})
iso_thick = iso_thick.transpose('time','sigma_mid','nlat','nlon')

htdiff = iso_thick.sum('sigma_mid') - (ds_temp['HT']/100.).assign_attrs({'units':'m'})
print("Max difference from true HT = {}m".format(abs(htdiff).max().values))

### 5. Compute Isopycnal Layer Depth

In [ ]:
# Cumulative sum of layer thickness yields depth of layer edges:
iso_depth = iso_thick.cumsum('sigma_mid').rename('iso_depth').rename({'sigma_mid':'sigma_bot'}).assign_attrs({'units':'m','long_name':'Isopycnal Layer Depth'})
sigma_bot = sigma_edge.isel(sigma=slice(1,None)).rename({'sigma':'sigma_bot'}).assign_attrs({'long_name':'Sigma2 at bottom of layer'})
iso_depth['sigma_bot'] = sigma_bot
iso_depth = iso_depth.transpose('time','sigma_bot','nlat','nlon')

In [ ]:
iso_depth

In [ ]:
iso_depth.isel(time=0,sigma_bot=84).plot(size=6,vmax=5500)

In [ ]:
# Isopycnal depth of bottom-most layer should be same as HT.
htdiff =  iso_depth.isel(sigma_bot=-1) - (ds_temp['HT']/100.).assign_attrs({'units':'m'})
print("Max difference from true HT = {}m".format(abs(htdiff).max().values))

### 6. Compute Isopycnal Layer Horizontal Volume Flux

In [ ]:
## Grid Metrics
dxu = ds_uvel['DXU']
dyu = ds_uvel['DYU']
dxt = ds_temp['DXT']
dyt = ds_temp['DYT']
dz = ds_temp['dz']
tarea = ds_temp['TAREA']

In [ ]:
u_e = ds_uvel['UVEL']
u_e = u_e.where(u_e<1.e30,0)
u_i = ds_uisop['UISOP'].drop(['TLONG','ULAT'])
u_i = u_i.where(u_i<1.e30,0)
u_s = ds_usubm['USUBM'].drop(['TLONG','ULAT'])
u_s = u_s.where(u_s<1.e30,0)
v_e = ds_vvel['VVEL']
v_e = v_e.where(v_e<1.e30,0)
v_i = ds_visop['VISOP'].drop(['ULONG','TLAT'])
v_i = v_i.where(v_i<1.e30,0)
v_s = ds_vsubm['VSUBM'].drop(['ULONG','TLAT'])
v_s = v_s.where(v_s<1.e30,0)

In [ ]:
# Grid-oriented Volume FLuxes:
u_e = (u_e*dyu*dz/1.e6).assign_attrs({'units':'m^3/s'})
v_e = (v_e*dxu*dz/1.e6).assign_attrs({'units':'m^3/s'})
u_i = (u_i*dyt*dz/1.e6).assign_attrs({'units':'m^3/s'})
v_i = (v_i*dxt*dz/1.e6).assign_attrs({'units':'m^3/s'})
u_s = (u_s*dyt*dz/1.e6).assign_attrs({'units':'m^3/s'})
v_s = (v_s*dxt*dz/1.e6).assign_attrs({'units':'m^3/s'})

In [ ]:
# Convert u_e,v_e to C-grid fluxes
u_e = 0.5*(u_e+u_e.shift(nlat=1))
v_e = 0.5*(v_e+v_e.roll(nlon=1,roll_coords=False))

In [ ]:
# Combine velocity components 
u = xr.concat([u_e,u_i,u_s],dim=ds_moctemp.moc_components)
v = xr.concat([v_e,v_i,v_s],dim=ds_moctemp.moc_components)

In [ ]:
%%time
# Volume fluxes in density-space. 
iso_uflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=u,dim=['z_t'],density=False)
iso_uflux = iso_uflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})

iso_vflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=v,dim=['z_t'],density=False)
iso_vflux = iso_vflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})

# Look for fix to histogram keep_coords=True to avoid this step in future:
iso_uflux = iso_uflux.assign_coords({'moc_components':ds_moctemp.moc_components})
iso_vflux = iso_vflux.assign_coords({'moc_components':ds_moctemp.moc_components})

In [ ]:
# Vertical sum in density-space should reproduce vertical sum in depth-space
ufluxdiff = iso_uflux.sum('sigma') - u.sum('z_t')
print("Max difference from true Uflux = {}".format(abs(ufluxdiff).max().values))
vfluxdiff = iso_vflux.sum('sigma') - v.sum('z_t')
print("Max difference from true Vflux = {}".format(abs(vfluxdiff).max().values))

#### The above large differences appear to be associated with the Nordic Sea overflow parameterization. I'm not sure how to handle overflow velocities in the MOC computation:

In [ ]:
ufluxdiff.isel(moc_comp=0).plot(size=7,vmin=-1.e5,vmax=1.e5)

### 7. Compute Vertical Volume Flux from horizontal flux convergence

In [ ]:
%%time
wflux = popmoc.wflux(iso_uflux,iso_vflux,'sigma',sigma_edge,grid='C')
wflux = wflux.assign_coords({'TLAT':ds['TLAT'],'TLONG':ds['TLONG']}).drop(['ULAT','ULONG'])

### 8. Define MOC region masks

In [ ]:
## Define the MOC region mask:
rmask = ds.REGION_MASK.drop(['ULONG','ULAT'])
rmaskglob = xr.where((rmask>0),1,0)
rmaskatl = xr.where((rmask>=6) & (rmask<=11),1,0)
rmaskmoc = xr.concat([rmaskglob,rmaskatl],dim=ds_moctemp.transport_regions)

In [ ]:
rmaskmoc.plot(levels=[0,1,2,3],col='transport_reg',size=5);

### 9. Compute MOC

In [ ]:
%%time
MOC = popmoc.compute_MOC(wflux,rmaskmoc,ds_moctemp.lat_aux_grid)
MOC = MOC.transpose('time','transport_reg','moc_comp','sigma','lat_aux_grid')

### 10. Add Southern Boundary Fluxes for Atlantic Region

In [ ]:
# determine j=index of Atlantic region southern boundary
tmp = rmaskmoc.isel(transport_reg=1).sum('nlon')
atl_j = 0
j = 0
while (atl_j==0):
    if (tmp.isel(nlat=j).data>0):
        atl_j = j
    j += 1
atl_j = atl_j - 1
atl_j

In [ ]:
rmaskmoc.coords['TLAT'][84]

In [ ]:
# add vflux at southern boundary of Atlantic domain
tmp = iso_vflux*(rmaskmoc.shift(nlat=-1))
tmp = tmp.isel(nlat=atl_j,transport_reg=1).sum('nlon')
moc_s = -tmp.sortby('sigma',ascending=False).cumsum('sigma').sortby('sigma',ascending=True)/1.e6
moc_s['sigma'] = sigma_edge.isel(sigma=slice(0,-1))
MOC[{'transport_reg':1}] = MOC[{'transport_reg':1}] + moc_s

In [ ]:
MOC.isel(time=0).isel(transport_reg=0,moc_comp=0).plot(ylim=[40,28])

In [ ]:
MOC.isel(time=0).isel(transport_reg=1,moc_comp=0).plot(ylim=[40,28])

### 11. Save to netcdf

In [ ]:
dsout = MOC.to_dataset()
dsout['iso_thick'] = iso_thick
dsout['iso_depth'] = iso_depth

In [ ]:
ds_out_MOC = xr.merge([MOC.rename('MOC')])
ds_out_MOC.attrs['description'] = 'Meridional overturning circulation (MOC)'
ds_out_MOC.attrs['units'] = 'Sv'
ds_out_MOC.attrs['author'] = 'Mauricio Rocha'
ds_out_MOC.attrs['email'] = 'mauricio.rocha@usp.br'

### Define k-index array

In [ ]:
dims = np.shape(temp)
#ne = dims[0] # ensember member
nt = dims[0]  # time
nz = dims[1]  # depth
ny = dims[2]  # latitude
nx = dims[3]  # longitude
kji = np.indices((nz,ny,nx))
kindices = kji[0,:,:,:] + 1 

#### Define sigma2_T

In [ ]:
refz = 2000 # reference depth
refdep = xr.full_like(salt,refz).rename('REFDEP')
# Sigma2 on model TLAT, TLONG
sigma2_T = pop_tools.eos(salt=salt,temp=temp,depth=refdep) - 1000
sigma2_T = sigma2_T.assign_attrs({'long_name':'Sigma referenced to {}m'.format(refz),'units':'kg/m^3'})
sigma2_T = sigma2_T.mean(dim=["time"]) # Average over time
# apply T-grid mask
#mask=kindices<=ds['KMT'].values[None,:,:]
#sigma2_T = sigma2_T.where(mask)

### Define target sigma-2 vertical grid

In [ ]:
# Use predefined 86-layer sigma2 grid:
sigma_mid,sigma_edge = popmoc.sigma2_grid_86L()

### Compute MOC(Sigma2) using xhistogram 

#### 1. Compute Isopycnal Layer Thickness

In [ ]:
# Here, test histogram by counting cells in each density bin. Vertical sum should be same as KMT.
iso_count = histogram(sigma2_T, bins=[sigma_edge.values],dim=['z_t'],density=False)
iso_count = iso_count.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
kmtdiff = (iso_count.sum('sigma') - ds_smbb_temp['KMT'].mean(dim=["time"]))
print("Max difference from true KMT = {}".format(abs(kmtdiff).max().values))

In [ ]:
# Use histogram to compute layer thickness. Vertical sum should be same as HT.
dzwgts = (ds_smbb_temp['dz']/100.).assign_attrs({'units':'m'})
dzwgts = dzwgts.mean(dim=["time"]) # Average over time
iso_thick = histogram(sigma2_T, bins=[sigma_edge.values], weights=dzwgts,dim=['z_t'],density=False)
iso_thick = iso_thick.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
iso_thick = iso_thick.rename('Isopycnal Layer Thickness').assign_attrs({'units':'m'})
htdiff = iso_thick.sum('sigma') - (ds_smbb_temp['HT']/100.).assign_attrs({'units':'m'})
htdiff = htdiff.mean(dim=["time"]) # Average over time
print("Max difference from true HT = {}m".format(abs(htdiff).max().values))
#In the original Notebook, the maximum difference is: HT = 1.2270752449694555e-05m

#### 2. Compute Isopycnal Layer Depth

In [ ]:
# Cumulative sum of layer thickness yields depth of layer edges:
iso_depth = iso_thick.cumsum('sigma').rename('Isopycnal Layer Depth')
iso_depth['sigma'] = sigma_edge.isel(sigma=slice(1,None))

In [ ]:
iso_depth.isel(sigma=84).plot(size=6,vmax=5500)

In [ ]:
# Isopycnal depth of bottom edge should be same as HT.
htdiff =  iso_depth.isel(sigma=-1) - (ds_smbb_temp['HT']/100.).assign_attrs({'units':'m'})
htdiff = htdiff.mean(dim=["time"]) # Average over time
print("Max difference from true HT = {}m".format(abs(htdiff).max().values))
#Max difference from true HT = 1.2270752449694555e-05m

#### 3. Compute Isopycnal Layer Horizontal Volume Flux

In [ ]:
# Grid-oriented Volume FLuxes:
uvel = uvel.where(uvel<1.e30).fillna(0.)
vvel = vvel.where(vvel<1.e30).fillna(0.)
uvel = (uvel*ds_smbb_uvel['DYU']*ds_smbb_uvel['dz']/1.e6).assign_attrs({'units':'m^3/s'})
vvel = (vvel*ds_smbb_vvel['DXU']*ds_smbb_vvel['dz']/1.e6).assign_attrs({'units':'m^3/s'})
uvel = uvel.mean(dim=["time"]) # Average over time
vvel = vvel.mean(dim=["time"]) # Average over time

In [ ]:
# Volume fluxes in density-space. Vertical sum is density-space should reproduce vertical sum in depth-space.
iso_uflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=uvel,dim=['z_t'],density=False) # The 'numpy.histogram_bin_edges' function is not implemented by Dask array.
iso_uflux = iso_uflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
iso_vflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=vvel,dim=['z_t'],density=False)
iso_vflux = iso_vflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})

ufluxdiff = iso_uflux.sum('sigma') - uvel.sum('z_t')
vfluxdiff = iso_vflux.sum('sigma') - vvel.sum('z_t')
print("Max difference from true Uflux = {}".format(abs(ufluxdiff).max().values))
print("Max difference from true Vflux = {}".format(abs(vfluxdiff).max().values))
#Max difference from true Uflux = 1367813.3765927013
#Max difference from true Vflux = 456888.7641561439

Need to investigate these differences, which appear to be associated with overflows. The difference plot below shows zero almost everywhere except near Nordic Seas overflow points.

In [ ]:
ufluxdiff.plot(size=7,vmin=-1.e5,vmax=1.e5)

#### 4. Compute Vertical Volume Flux using model divergence operator

In [ ]:
wflux = popmoc.pop_isowflux(iso_uflux,iso_vflux,'sigma',sigma_edge)

#### 5. Compute Zonal Sums of Vertical Volume Flux in latitude strips

In [ ]:
# Load predefined 1-degree target latitude grid:
lat_mid,lat_edge = popmoc.latitude_grid_1deg()

In [ ]:
## Define MOC region mask with legend:
rmask = ds_smbb_temp.REGION_MASK
rmask=rmask.mean(dim=["time"])
rmaskmoc = rmask.where(rmask>0)
rmaskmoc = xr.where((rmask>0),1,rmaskmoc)
rmaskmoc = xr.where((rmask>=6) & (rmask<=11),2,rmaskmoc)
rmaskmoc.plot(levels=[0,1,2,3]);
rmaskmoc.attrs['legend'] = {0:"Global",1:"IndoPac+SO",2:"Atlantic"}

In [ ]:
tarea = ds_smbb_temp['TAREA']
tarea=tarea.mean(dim=["time"])
tlat = ds_smbb_temp['TLAT']
wflux_zonsum = popmoc.mesh_zonalavg(wflux,tarea,tlat,rmaskmoc,rmaskmoc.legend,lat_edge,sum=True)

#### 6. Compute cumulative meridional integral of zonally-summed wflux

A southward cumulative integral from 90N avoids issues associated with southern boundary of Atlantic region.

In [ ]:
moc = -wflux_zonsum.sel(lat=slice(None,None,-1)).cumsum('lat').sel(lat=slice(None,None,-1))
moc = (moc/1.e6).assign_attrs({'units':'Sv'})   
moc.name = 'MOC'

In [ ]:
moc.isel(region=0).plot(size=7,vmax=40,levels=21)
plt.ylim([38,29])

In [ ]:
moc.isel(region=1).plot(size=7,vmax=40,levels=21)
plt.ylim([38,29])

In [ ]:
moc.isel(region=2).plot(size=7,vmax=40,levels=21)
plt.ylim([38,29])

In [ ]:
moc.isel(region=[1,2]).sum('region').plot(size=7,vmax=40,levels=21)
plt.ylim([38,29])